# Generate instrument sector information using dedicated service - Python

### Overview
Reference data encompasses a wide range of specification data about:
* financial instrument such as asset class, symbol, maturity, etc.
* counterparties such as issuer information.
* pricing such as Open, high low and close prices.

The challenge with reference data is that it tends to be sourced from multiple sources: internal, counterparties and providers.

Reference data dedicated service is a result of various data sources and fields sourcing, validation, cross-checking and normalization.<br>

This sample requests **on-demand** exchange data and enables fields mapping by calling a dedicated static data service, making available clean data.

### Inputs/outputs
Exchange data sample is designed to help you searching and requesting exchange reference data by using a wide range of commonly used fields.

It takes exchange identifier as per input filter and returns the *entries* matching the request:
* List of instruments' identifiers
* Countries
* Currencies 
* Tick size rules
* etc. 

### Services used
This sample uses *gRPC requests* in order to retrieve sectors reference data from the hosted service. The queried endpoint in this script are:
* *StaticSectorService*: to directly retrieve sector information objects from the server

### Modules required
1. Systemathics packages:
    * *systemathics.apis.services.sector_data.v1*
2. Open source packages
    * *googleapis-common-protos*
    * *protobuf*
    * *grpcio*
    * *pandas*
    
***

# Run Exchange data sample

### Step 1: Install packages and import them

In [ ]:
pip install googleapis-common-protos protobuf grpcio pandas systemathics.apis

In [ ]:
import os
import grpc
import pandas as pd
import google.protobuf as pb
import systemathics.apis.services.static_data.v1.static_sector_pb2 as static_sector
import systemathics.apis.services.static_data.v1.static_sector_pb2_grpc as static_sector_service

### Step 2: Prepare API requests
The following code snippets retrieve authentication token to be used in upcomming API requests:

In [ ]:
token = f"Bearer {os.environ['AUTH0_TOKEN']}"
display(token)

### Step 3: Define necessary functions
In this part we define a function dedicated to retrieve **sectors data** for a given **classification** and **code**.
This snippet will be used in the next paragraphs for examples

In [ ]:
def get_sector_data(provider, code):
    
    # generate sector data request
    request = static_sector.StaticSectorRequest( 
        provider = provider,
        code = code
    )
    try:
        with open(os.environ['SSL_CERT_FILE'], 'rb') as f:
            credentials = grpc.ssl_channel_credentials(f.read())
        with grpc.secure_channel(os.environ['GRPC_APIS'], credentials) as channel:

            # instantiate the sector data service
            service = static_sector_service.StaticSectorServiceStub(channel)

            # process the request
            response = service.StaticSector(
                request = request, 
                metadata = [('authorization', token)]
            )
        return response
    except grpc.RpcError as e:
        display(e.code().name)
        display(e.details())


### Step 4: Retrieve sectors data

#### 4.1: TRBC

In [ ]:
# set the exchange to retrieve equities data
provider = 'TRBC'
code = '534020'

In [ ]:
trbc = get_sector_data(provider, code)
trbc.data

#### 4.2: ICB

In [ ]:
# set the exchange to retrieve equities data
provider = 'ICB'
code = '6575'

In [ ]:
icb = get_sector_data(provider, code)
icb.data

#### 4.3: SIC

In [ ]:
# set the exchange to retrieve equities data
provider = 'SIC'
code = '3812'

In [ ]:
sic = get_sector_data(provider, code)
sic.data

#### 4.4: BBID

In [ ]:
# set the exchange to retrieve equities data
provider = 'BBID'
code = '68966480'

In [ ]:
bbid = get_sector_data(provider, code)
bbid.data

#### 4.5: GICS

In [ ]:
# set the exchange to retrieve equities data
provider = 'GICS'
code = '3520'

In [ ]:
gics = get_sector_data(provider, code)
gics.data